In [27]:
import numpy as np
import pandas as pd

In [19]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [28]:
glove = pd.read_table('glove.6B.50d.txt', sep=' ', header=None, quoting=3, index_col=0)

In [13]:
glove.head()

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
0,,,,,,,,,,,,,,,,,,,,,
the,0.418000,0.249680,-0.41242,0.12170,0.34527,-0.044457,-0.49688,-0.17862,-0.00066,-0.656600,...,-0.298710,-0.157490,-0.347580,-0.045637,-0.44251,0.187850,0.002785,-0.184110,-0.115140,-0.78581
",",0.013441,0.236820,-0.16899,0.40951,0.63812,0.477090,-0.42852,-0.55641,-0.36400,-0.239380,...,-0.080262,0.630030,0.321110,-0.467650,0.22786,0.360340,-0.378180,-0.566570,0.044691,0.30392
.,0.151640,0.301770,-0.16763,0.17684,0.31719,0.339730,-0.43478,-0.31086,-0.44999,-0.294860,...,-0.000064,0.068987,0.087939,-0.102850,-0.13931,0.223140,-0.080803,-0.356520,0.016413,0.10216
of,0.708530,0.570880,-0.47160,0.18048,0.54449,0.726030,0.18157,-0.52393,0.10381,-0.175660,...,-0.347270,0.284830,0.075693,-0.062178,-0.38988,0.229020,-0.216170,-0.225620,-0.093918,-0.80375
to,0.680470,-0.039263,0.30186,-0.17792,0.42962,0.032246,-0.41376,0.13228,-0.29847,-0.085253,...,-0.094375,0.018324,0.210480,-0.030880,-0.19722,0.082279,-0.094340,-0.073297,-0.064699,-0.26044


In [30]:
unique_tokens = pd.read_table('unique_tokens.dmp', header=None)

In [37]:
unique_tokens.head(1000)

,0
0,NaN
1,\n
2,!
3,""""
4,'
5,'a
6,'aah
7,'aamye
8,'ab
9,'aba


In [ ]:
id2word = {}
word2id = {}
for i, word in enumerate(unique_tokens.loc[:, 0].tolist()):
    id2word[i] = word
    word2id[word] = i

In [2]:
train = pd.read_table('train.dmp', sep='|')

In [3]:
train.shape

(1660, 11)

In [4]:
train.head()

,0,1,2,3,4,5,6,7,8,9,10
0,i,was,out,with,the,morning,sun,\n,couldn,'t,sleep
1,was,out,with,the,morning,sun,\n,couldn,'t,sleep,","
2,out,with,the,morning,sun,\n,couldn,'t,sleep,",",so
3,with,the,morning,sun,\n,couldn,'t,sleep,",",so,i
4,the,morning,sun,\n,couldn,'t,sleep,",",so,i,thought


In [ ]:
n_samples = train.shape[0]

In [6]:
input_length = train.shape[1] - 1

In [8]:
vocab_size = len(unique_tokens)

In [ ]:
train_tokens = train.replace(word2id)

In [ ]:
x = train_tokens.iloc[:, :input_length].values
y_tmp = train_tokens.iloc[:, [input_length]].values

In [ ]:
y = np.zeros((train.shape[0], vocab_size))
y[list(range(n_samples)), y_tmp[:, 0]] = 0

In [ ]:
n_train_samples = int(n_samples * .7)

In [ ]:
train_index = set(np.random.choices(list(range(n_samples)), n_train_samples, replace=False))

In [ ]:
valid_index = set(list(range(n_samples))) - train_index

In [ ]:
train_x = x[train_index, :]
train_y = y[train_index, :]
valid_x = x[valid_index, :]
valid_y = y[valid_index, :]

In [23]:
test = np.zeros((3, 3))

In [25]:
test[[0, 1], [1, 2]] = 1

In [26]:
test

array([[ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  0.]])

In [9]:
emb_dim = 50

In [7]:
model = Sequential()

In [10]:
model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length))
model.add(LSTM(1000, activation='relu', return_sequences=True))
model.add(LSTM(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 50)            50000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 1000)          4204000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 1000)              8004000   
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              1001000   
Total params: 14,260,000
Trainable params: 14,260,000
Non-trainable params: 0
_________________________________________________________________
